# Naver 채권분석리포트 Crawling

## 1. 날짜, 증권사, 제목, pdf 다운받기

In [ ]:
# PyPDF2나 pdfminer를 찾을 수 없다고 한다면, 아래 주석을 풀고 install을 시도하세요.
# pip install PyPDF2 
# pip install pdfminer.six

# moduel import
import requests
from bs4 import BeautifulSoup
import os
import urllib.request
from PyPDF2 import PdfFileReader
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io

# 함수 정의
def BondReportsCrawl():
    pdf_list = [] # pdf의 날짜, 증권사, 제목을 담을 빈 리스트 생성
    BASE_DIR = os.getcwd() # 현재 작업하고 있는 곳을 BASE_DIR로 정의
    REPORTS_DIR = BASE_DIR # REPORTS_DIR = BASE_DIR
    
    # Naver의 채권분석리포트의 마지막 페이지는 116이므로 117까지 for문 돌릴 준비
    for i in range(1, 117):
        print(i,'번 페이지에 있는 문서를 가져오는 중입니다.') # 작업 상황 확인을 위함 
        url = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(i) # page={}에 i 대입
        response = requests.get(url) # url로 requests를 보냄 
        soup = BeautifulSoup(response.content, 'html.parser') # soup 생성
        reports = soup.find('table', class_='type_1').find_all('tr')[1:] # 찾아야할 content 범위 설정

        # 날짜, 증권사, 제목, pdf 다운
        for td in reports:
            filename = "" # filename 을 만들 빈 문자열 생성, for문이 돌 때마다 초기화

            # date 코드
            date = td.find('td', class_='date')
            if date is not None: # None 값들이 검사되므로 if문 추가
                date = date.text # text만 추출
                filename += date # filename 에 추가
                filename += '_' # 다음에 들어올 author와의 구분을 위한 _ 추가 

            # author 코드
            author = td.find_all('td') 
            if len(author) == 5: # 길이를 확인해보니, len이 5인 것만 정상적으로 증권사 이름들이 추출됨 
                author = author[1].text
                filename += author
                filename += '_'  # 다음에 들어올 title와의 구분을 위한 _ 추가 

            # title 코드 
            title = td.find('td').find('a')
            if title is not None: # None 값들이 검사되므로 if문 추가
                title = title.text
                filename += title # filename 에 추가 
                # computer가 읽을 수 없는 특수문자들을 대체 (후에 정규표현식으로 고칠 것)
                # 공백제거 코드도 추가해 줄 것 
                filename = filename.replace('?','').replace('.','').replace('!','').replace('/','')
                filename = filename.replace(':','').replace(';','').replace('=','').replace('+','')
                filename = filename.replace('-','').replace('*','').replace('"','').replace('<','')
                filename = filename.replace('>','').replace('[','').replace(']','')
                filename = filename + '.pdf'
                pdf_list.append(filename) # pdf_list 에 filename을 추가함 

            # pdf 다운받기 코드
            pdf_href = td.find('td', class_='file')
            if pdf_href is not None: # None값이 있기 때문에 if문 추가
                pdf_href = str(pdf_href.find('a')['href'])
                pdf_url = pdf_href
                print(pdf_url) # pdf_url 출력해서 제대로 된 url인지 확인

                # 가장 중요한 부분. requests.get을 활용해도 pdf 파일을 다운 받을 수 있다!!! 
                r = requests.get(pdf_url) # url로 requests 보냄
                naver_reports_path = os.path.join(REPORTS_DIR, filename) # 작업하고 있는 폴더에 파일 추가

                with open(naver_reports_path, 'wb') as f: # 파일 쓰기
                    f.write(r.content)

    print(pdf_list)
    print("총 가져온 pdf파일 개수: ", len(pdf_list))
    
BondReportsCrawl()

## 2. pdf -> txt 파일로 변환

In [ ]:
# pdf -> txt 로 변환하는 함수                
def pdfparser(data):
    ccount = 0
    BASE_DIR = os.getcwd()
    REPORTS_DIR = BASE_DIR
    
    try: # 오류가 발생하는 pdf 들이 존재하기 때문에, try문 사용
        fp = open(data, 'rb')
        rsrcmgr = PDFResourceManager()
        retstr = io.StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        for page in PDFPage.get_pages(fp):
            interpreter.process_page(page)
            data =  retstr.getvalue()
            text_path = os.path.join(REPORTS_DIR, pdf_list[i]+'.txt')
            with open(text_path, 'w', encoding='utf-8') as f:
                f.write(data)

    except AttributeError:
        ccount += 1
        print(pdf_list[i],'는 Error로 인해 pass합니다.')
        print("현재까지의 누적 Errors: ", ccount)
        pass
    
# 이 작업을 통해, pdf_list에서 날짜, 증권사, 제목을 확인할 수 있고,
# pdf 파일을 얻을 수 있다.
# 또한, pdf 파일을 txt로 변환하는 함수까지 정의해 놓은 상태
# 아래에 있는 코드를 실행하면, pdf를 txt로 변환하기 시작합니다.

In [ ]:
## 시간이 꽤 오래 걸립니다...!!
# pdf -> txt 로 변환하는 code
# 오류가 나면, range()를 수정하면서 변환
for i in range(1, (len(pdf_list)+1)):
    print(i)
    print(pdf_list[i], '를 txt로 변환 중입니다...') 
    pdfparser(pdf_list[i])

In [2]:
# PyPDF2나 pdfminer를 찾을 수 없다고 한다면, 아래 주석을 풀고 install을 시도하세요.
# pip install PyPDF2 
# pip install pdfminer.six

# moduel import
import requests
from bs4 import BeautifulSoup
import os
import urllib.request
from PyPDF2 import PdfFileReader
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io

# 함수 정의
def BondReportsCrawl():
    count = 0
    pdf_list = [] # pdf의 날짜, 증권사, 제목을 담을 빈 리스트 생성
    BASE_DIR = os.getcwd() # 현재 작업하고 있는 곳을 BASE_DIR로 정의
    REPORTS_DIR = BASE_DIR # REPORTS_DIR = BASE_DIR
    
    # Naver의 채권분석리포트의 마지막 페이지는 116이므로 117까지 for문 돌릴 준비
    for i in range(1, 117):
        print(i,'번 페이지에 있는 문서를 가져오는 중입니다.') # 작업 상황 확인을 위함 
        url = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(i) # page={}에 i 대입
        response = requests.get(url) # url로 requests를 보냄 
        soup = BeautifulSoup(response.content, 'html.parser') # soup 생성
        reports = soup.find('table', class_='type_1').find_all('tr')[1:] # 찾아야할 content 범위 설정

        # 날짜, 증권사, 제목, pdf 다운
        for td in reports:
            filename = "" # filename 을 만들 빈 문자열 생성, for문이 돌 때마다 초기화

            # date 코드
            date = td.find('td', class_='date')
            if date is not None: # None 값들이 검사되므로 if문 추가
                date = date.text # text만 추출
                filename += date # filename 에 추가
                filename += '_' # 다음에 들어올 author와의 구분을 위한 _ 추가
                count += 1
                
    return count
                
BondReportsCrawl()

1 번 페이지에 있는 문서를 가져오는 중입니다.
2 번 페이지에 있는 문서를 가져오는 중입니다.
3 번 페이지에 있는 문서를 가져오는 중입니다.
4 번 페이지에 있는 문서를 가져오는 중입니다.
5 번 페이지에 있는 문서를 가져오는 중입니다.
6 번 페이지에 있는 문서를 가져오는 중입니다.
7 번 페이지에 있는 문서를 가져오는 중입니다.
8 번 페이지에 있는 문서를 가져오는 중입니다.
9 번 페이지에 있는 문서를 가져오는 중입니다.
10 번 페이지에 있는 문서를 가져오는 중입니다.
11 번 페이지에 있는 문서를 가져오는 중입니다.
12 번 페이지에 있는 문서를 가져오는 중입니다.
13 번 페이지에 있는 문서를 가져오는 중입니다.
14 번 페이지에 있는 문서를 가져오는 중입니다.
15 번 페이지에 있는 문서를 가져오는 중입니다.
16 번 페이지에 있는 문서를 가져오는 중입니다.
17 번 페이지에 있는 문서를 가져오는 중입니다.
18 번 페이지에 있는 문서를 가져오는 중입니다.
19 번 페이지에 있는 문서를 가져오는 중입니다.
20 번 페이지에 있는 문서를 가져오는 중입니다.
21 번 페이지에 있는 문서를 가져오는 중입니다.
22 번 페이지에 있는 문서를 가져오는 중입니다.
23 번 페이지에 있는 문서를 가져오는 중입니다.
24 번 페이지에 있는 문서를 가져오는 중입니다.
25 번 페이지에 있는 문서를 가져오는 중입니다.
26 번 페이지에 있는 문서를 가져오는 중입니다.
27 번 페이지에 있는 문서를 가져오는 중입니다.
28 번 페이지에 있는 문서를 가져오는 중입니다.
29 번 페이지에 있는 문서를 가져오는 중입니다.
30 번 페이지에 있는 문서를 가져오는 중입니다.
31 번 페이지에 있는 문서를 가져오는 중입니다.
32 번 페이지에 있는 문서를 가져오는 중입니다.
33 번 페이지에 있는 문서를 가져오는 중입니다.
34 번 페이지에 있는 문서를 가져오는 중입니다.
35 번 페이지에 있는 문서를 가져오는 중입니다.
36 번 페이지에 있는 문서를 가져오는 중입니다.
3

3466

In [3]:
3466 - 3370

96